In [0]:
!pip install music21==5.7.2

In [0]:
import music21
music21.__version__

'5.7.2'

In [0]:
import os
import json
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import *
from music21 import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Using TensorFlow backend.


In [0]:
data_directory = './drive/My Drive/'
data_file = 'Data_Tunes.txt'
charIndex_json = 'char2index.json'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [0]:
def preprocess(data):

  #The data has some unnecessary data before each song and has a few lines seperating each song, the 'Z' label is used used to indicate the begining of each song.
  list1 = list(data)
  list2 = ['start_label']
  #This indicates the start of each song
  ignore_chars = ['X', 'T', 'M', 'K', 'S', 'P']
  i=0
  while(i<len(list1)):
    if(((list1[i] in ignore_chars) and (list1[i+1]==':')) or list1[i]=='%'):
      del list2[-1]
      while(list1[i] != '\n'):
        i = i+1
    if(list1[i] =='\n'):
      list2.append("start_label")
    else:

      list2.append(list1[i])
    
    i = i+1
  i = 0
  # Z -Character
  preprocess_data = []
  while (i<len(list2)):
    if (list2[i]=='start_label' ):
      preprocess_data.append('Z')
      i = i + 1
    else:
      preprocess_data.append(list2[i])
      i = i + 1
#Data seperated and reruened as a list after being read from the file
  return preprocess_data

file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
preprocess_data = preprocess(data)
print(preprocess_data)

['Z', '{', '"', ' ', '"', ':', ' ', '0', ',', ' ', '"', '!', '"', ':', ' ', '1', ',', ' ', '"', '\\', '"', '"', ':', ' ', '2', ',', ' ', '"', '#', '"', ':', ' ', '3', ',', ' ', '"', "'", '"', ':', ' ', '4', ',', ' ', '"', '(', '"', ':', ' ', '5', ',', ' ', '"', ')', '"', ':', ' ', '6', ',', ' ', '"', '+', '"', ':', ' ', '7', ',', ' ', '"', ',', '"', ':', ' ', '8', ',', ' ', '"', '-', '"', ':', ' ', '9', ',', ' ', '"', '/', '"', ':', ' ', '1', '0', ',', ' ', '"', '0', '"', ':', ' ', '1', '1', ',', ' ', '"', '1', '"', ':', ' ', '1', '2', ',', ' ', '"', '2', '"', ':', ' ', '1', '3', ',', ' ', '"', '3', '"', ':', ' ', '1', '4', ',', ' ', '"', '4', '"', ':', ' ', '1', '5', ',', ' ', '"', '5', '"', ':', ' ', '1', '6', ',', ' ', '"', '6', '"', ':', ' ', '1', '7', ',', ' ', '"', '7', '"', ':', ' ', '1', '8', ',', ' ', '"', '8', '"', ':', ' ', '1', '9', ',', ' ', '"', '9', '"', ':', ' ', '2', '0', ',', ' ', '"', ':', '"', ':', ' ', '2', '1', ',', ' ', '"', '=', '"', ':', ' ', '2', '2', ',', ' '

In [0]:
def read_data(preprocessed_data):
  #In this function we create dictionaries to identify notes to indices
  char2index = {ch:i for i,ch in enumerate(sorted(list(set(preprocess_data))))}
  #writing to file, we need it later!
  with open(os.path.join(data_directory,charIndex_json), mode = 'w') as f:
    json.dump(char2index,f)
  index2char = {i:ch for (ch,i) in char2index.items()}
  vocab_size = len(char2index)
  all_char_as_index = np.asarray([char2index[x] for x in preprocessed_data ])
  return all_char_as_index, vocab_size

all_characters_as_indices, num_unique_chars = read_data(preprocess_data)
print(read_data(preprocess_data))

(array([34, 58,  2,  0,  2, 21,  0, 11,  8,  0,  2,  1,  2, 21,  0, 12,  8,
        0,  2, 36,  2,  2, 21,  0, 13,  8,  0,  2,  3,  2, 21,  0, 14,  8,
        0,  2,  4,  2, 21,  0, 15,  8,  0,  2,  5,  2, 21,  0, 16,  8,  0,
        2,  6,  2, 21,  0, 17,  8,  0,  2,  7,  2, 21,  0, 18,  8,  0,  2,
        8,  2, 21,  0, 19,  8,  0,  2,  9,  2, 21,  0, 20,  8,  0,  2, 10,
        2, 21,  0, 12, 11,  8,  0,  2, 11,  2, 21,  0, 12, 12,  8,  0,  2,
       12,  2, 21,  0, 12, 13,  8,  0,  2, 13,  2, 21,  0, 12, 14,  8,  0,
        2, 14,  2, 21,  0, 12, 15,  8,  0,  2, 15,  2, 21,  0, 12, 16,  8,
        0,  2, 16,  2, 21,  0, 12, 17,  8,  0,  2, 17,  2, 21,  0, 12, 18,
        8,  0,  2, 18,  2, 21,  0, 12, 19,  8,  0,  2, 19,  2, 21,  0, 12,
       20,  8,  0,  2, 20,  2, 21,  0, 13, 11,  8,  0,  2, 21,  2, 21,  0,
       13, 12,  8,  0,  2, 22,  2, 21,  0, 13, 13,  8,  0,  2, 23,  2, 21,
        0, 13, 14,  8,  0,  2, 24,  2, 21,  0, 13, 15,  8,  0,  2, 25,  2,
       21,  0, 13, 16,  

In [0]:
#Reads the files and gives us what we need later.(We get the dictionaries we need)
def return_dict():
  read_data(preprocess_data)
  with open(os.path.join(data_directory, charIndex_json)) as f:
    char_to_index = json.load(f)
  index_to_char = {i:ch for ch, i in char_to_index.items()}
  num_unique_chars = len(index_to_char)
  return char_to_index , index_to_char , num_unique_chars


In [0]:
def data_create(all_char_as_index, vocab_size):
  total_len = len(all_char_as_index)
  num_examples = total_len//SEQ_LENGTH #// : to ensure we get and integer value ie 4 not 4.0
  X = np.zeros((num_examples,SEQ_LENGTH))#This is indexed, gets encoded by tf
  Y = np.zeros((num_examples,SEQ_LENGTH,vocab_size ))#This is one hot encoded

  for  i in range(num_examples):
    for j in range(SEQ_LENGTH):
      X[i,j] = all_char_as_index[i*SEQ_LENGTH + j]
      #One hot encoding
      Y[i,j, all_char_as_index[i*SEQ_LENGTH + j+1]] = 1

  return X,Y



In [0]:
def model_init(seq_length, vocab_size, Emb_dim=512, units=256):
  model = Sequential()
  #This is the training model
  model.add(Embedding(input_dim= vocab_size, output_dim= Emb_dim, input_shape = (seq_length,)))
#Each char is passed and trained against the following char
#Note on every layer we pass THE WHOLE SEQUENCE not just the last one
  model.add(LSTM(units, return_sequences = True))
  model.add(Dropout(0.2))

  model.add(LSTM(units, return_sequences = True))
  model.add(Dropout(0.2))

  model.add(LSTM(units, return_sequences = True))
  model.add(Dropout(0.2))

  model.add(TimeDistributed(Dense(vocab_size,activation='softmax')))

    
  return model


In [0]:
def make_model(num_unique_chars,Emb_dim = 512,units = 256):
  model = Sequential()
#This is the sampling model
  model.add(Embedding(input_dim= num_unique_chars, output_dim= Emb_dim, batch_input_shape = (1,1)))
#stateful ensures the previous y value gets passed as the next x value

  model.add(LSTM(units, return_sequences = True, stateful= True))

  model.add(Dropout(0.2))

  model.add(LSTM(units, return_sequences = True, stateful= True))
  model.add(Dropout(0.2))

  model.add(LSTM(units, return_sequences = True, stateful = True))
  model.add(Dropout(0.2))

  model.add(TimeDistributed(Dense(num_unique_chars,activation = 'softmax')))

    
  return model    

In [0]:
def generate_sequence(gen_seq_length):
  char_to_index, index_to_char , num_unique_chars = return_dict()

  model = make_model(num_unique_chars)
  model.load_weights('./weights.80.hdf5')
#Every song begins with 'Z'
  sequence_index = [char_to_index['Z']]

  for _ in range(gen_seq_length):
    #Each char is passed and the following char is obtained
    batch = np.zeros((1,1))
    batch[0,0] = sequence_index[-1]
    # We select the following note based on the probability distribution at random 
    predicted_probs = model.predict_on_batch(batch).ravel()
    sample = np.random.choice(range(num_unique_chars), size = 1, p = predicted_probs)

    sequence_index.append(sample[0])

  seq = ''.join(index_to_char[c] for c in sequence_index)

  seq = 'M:6/8\n' + str(seq)

  return seq

In [0]:
#Music 21 stuff
def convert_to_midi(abc):
  c = converter.subConverters.ConverterABC()
  c.registerOutputExtensions = ('midi', )
  c.parseData(abc)
  s = c.stream
  s.write('midi', fp = 'MiniProject.mid')

In [0]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()

preprocess_data = preprocess(data)

all_characters_as_indices, num_unique_chars = read_data(preprocess_data)

X,Y = data_create(all_characters_as_indices, num_unique_chars)

In [0]:
model = model_init(SEQ_LENGTH, num_unique_chars)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 64, 512)           31744     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64, 256)           787456    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64, 256)           525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64, 256)           525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 64, 256)          

In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

checkpoint = ModelCheckpoint(filepath='weights.{epoch:02d}.hdf5', monitor = 'loss', save_best_only=True, save_weights_only=True, period = 1)

model.fit(X,Y,batch_size = BATCH_SIZE, epochs = 80,callbacks = [checkpoint])

music = generate_sequence(192)

convert_to_midi(music)